In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import Boson
import Proca as Proca
from scipy.integrate import solve_ivp
Boson.general()
Proca.general()

In [ ]:
#Primero un caso sencillo con una estrella de bosones 1D.
LambT = 1  # autointeracción: 1 (repulsiva), 0, -1 (atractiva)
nodos = 1 #Numero de nodos, solo estamos en 1D por lo tanto basta con uno
p0x, p1x = 1, 0  # valores inciliales de sigma y de su primer derivada
u1x = 0  #Primera derivada del potencial

Iniciales = [p0x, p1x, u1x ] 
rmax = 25  #radio máximo

#Semilla para el potencial
Uxint = [0.1, 3]
datos = []
U0x,rTmax, _  = Boson.Bosshot(Iniciales, Uxint, rmax, LambT=LambT,
                                                n=nodos, lim=1e-8, info=False, klim=1500, outval=10, delta=0.2)

# plot the result
U0 = [p0x, p1x, U0x, u1x]
Boson.plotPerfBos(U0, rTmax, LambT)

In [ ]:
#Ya podemos activar los demás nodos
LambT = 0  # autointeracción: 1 (repulsive), 0 (without selinteraction), -1 (attractive)

nx, ny, nz = 0, 3, 0  # node numbers: nx -> nodes of \sigma_x, ny  -> nodes of \sigma_y, ...
nodos = [nx, ny, nz]  # list of node info


p0x, p1x = 0.6539145068038972, 0  # The central amplitude of \sigma_x (p0x) and its first derivative value (p1x), the latter is zero
p0y, p1y =  0.16875, 0  # The same in y-axes
p0z, p1z = 0, 0      # The same in z-axes
u1x, u1y, u1z = 0, 0, 0  # The first derivatives of the x, y, and z effective potential components are all zero.

Ini0 = [p0x, p1x, p0y, p1y, p0z, p1z, u1x, u1y, u1z]  # boundary conditions list
rmax = 50  # iteration radius
# Range of values used as seed for the effective potential
Uxint = [0.1, 2]
Uyint = [0.1, 2]
Uzint = [0.1, 2]

datos = []
U0x, U0y, U0z, rTmax, _, _, _ = Proca.Shoot_Proca1(Ini0, Uxint, Uyint, Uzint, rmax, LambT=LambT,
                                                nodos=nodos, lim=1e-8, info=False, klim=1500, outval=10, delta=0.2)

# plot the result
U0 = [p0x, p1x, p0y, p1y, p0z, p1z, U0x, u1x, U0y, u1y, U0z, u1z]
Proca.plotPerf(U0, rTmax, LambT)


In [ ]:
#Ahora podemos refinar la solución 
rext = 6  #Cuanto queremos incrementear el radio 
rmin, rmax = 0, rTmax + rext
limit = [rmin, rmax]
p1x, p1y, p1z, u1x, u1y, u1z = 0, 0, 0, 0, 0, 0  #Las condiciones de frontera
uk = [U0x, U0y, U0z]  #nuestras semillas de la solución no refinada

# solving
V0X = [p0x, p1x, p0y, p1y, p0z, p1z, None, u1x, None, u1y, None, u1z]
V0Duk = np.zeros(12*3)
#La posición de las derivadas no triviales
V0Duk[6] = 1
V0Duk[20] = 1
V0Duk[-2] = 1
V0 = np.concatenate((V0X, V0Duk))

#Posción de las cks que vamos a ajustar
indck = np.concatenate(([False, False, False, False, False, False, True, False, True, False, True, False], [False]*36))
V0[indck] = uk

BCind = [0, 0, 0]

indXc = np.array([False]*48) 
indXc[:5:2] = True  #La posción de las variables para las que queremos resolver.

inddXc = np.array([False]*48) #Las derivadas de las variables y sus posiciones
inddXc[12::12] =  True
inddXc[14::12] =  True
inddXc[16::12] =  True

V0fit = Proca.fitting(Proca.system_refinado, V0, indck, indXc, BCind, inddXc, limit, argf=LambT, tol=1e-10)
U0 = V0fit[:12]
Proca.plotPerf(U0, rmax, LambT, px=True, lim=False)

In [ ]:
#Ahora podemos pasar al calculo de la masa y la energía.
confi =[0.9185797787133904, 0.0, 0.0, 20, [1.0, 0.0, 0], 0] #Esta es la configuración general de la estrella

px0, py0, pz0 = confi[4]
rmin, rmax = 0, confi[3]
LambT = confi[-1] 

metodo = 'RK45'; Rtol = 1e-09; Atol = 1e-10
Nptos = 500; rspan = np.linspace(rmin, rmax, Nptos); 

arg = [LambT]
u0x, u0y, u0z = confi[0], confi[1], confi[2]
U0 = [px0, 0, py0, 0, pz0, 0, u0x, 0, u0y, 0, u0z, 0]

sol2 = solve_ivp(Proca.systemProca, [rmin, rmax], U0, t_eval=rspan,
                     args=(arg,), method=metodo, rtol=Rtol, atol=Atol)
rad = sol2.t
px, ux = sol2.y[0], sol2.y[6][0]
py, uy = sol2.y[2], sol2.y[8][0]
pz, uz = sol2.y[4], sol2.y[10][0]

sigtot = px**2

masaT =4*np.pi*Proca.Calc_Masa(rad, sigtot)
print(f"La masa es: {masaT}" ) 

In [ ]:
#Los datos de ejemplo para comprobar funcionamiento
#Datos para conectar [0,0,0] con [0,1,0]. ux,uy,uz,rmax,[p0],lambT
datos = [
    [0.9185797787133904, 0.0, 0.0, 20, [1.0, 0.0, 0], 0],
    [0.9096109737806662, 1.4608345383430958, 0.0, 23, [0.99, 0.025, 0], 0],
    [0.9075085207455894, 1.4574355805220829, 0.0, 20.801145652799903, [0.987, 0.05, 0], 0],
    [0.8975962120277898, 1.44147276871182, 0.0, 21.093870287124965, [0.975, 0.075, 0], 0],
    [0.8854098143064014, 1.421840747957605, 0.0, 20, [0.96, 0.1, 0], 0],
    [0.860034412595141, 1.3809870546640286, 0.0, 23, [0.93, 0.125, 0], 0],
    [0.8352695283628023, 1.3410911542449409, 0.0, 20, [0.9, 0.15, 0], 0],
    [0.8021225992772414, 1.2876610797360635, 0.0, 23, [0.86, 0.175, 0], 0],
    [0.7608283331922263, 1.2210832479554825, 0.0, 20, [0.81, 0.2, 0], 0],
    [0.702859111596206, 1.1275343620787186, 0.0, 23, [0.74, 0.225, 0], 0],
    [0.6205277161841714, 0.9945492936294549, 0.0, 20, [0.64, 0.25, 0], 0],
    [0.5628515737223025, 0.9012007021170526, 0.0, 23, [0.57, 0.26, 0], 0],
    [0.5135960474191803, 0.8213994220698124, 0.0, 27.154850032037178, [0.51, 0.265, 0], 0],
    [0.4727913155779859, 0.7551935214469294, 0.0, 28.011943689000802, [0.46, 0.267, 0], 0],
    [0.415531125359361, 0.6621824517247614, 0.0, 33, [0.39, 0.265, 0], 0],
    [0.3593021759210128, 0.5705465319937473, 0.0, 35, [0.32, 0.26, 0], 0],
    [0.2968719876911216, 0.46832403077246226, 0.0, 35, [0.24, 0.25, 0], 0],
    [0.2587505388574897, 0.40568756157789704, 0.0, 35, [0.19, 0.24, 0], 0],
    [0.22656416357286724, 0.352489658409764, 0.0, 35, [0.145, 0.23, 0], 0],
    [0.20031223041291482, 0.3088944767913174, 0.0, 39, [0.105, 0.22, 0], 0],
    [0.17818483844444952, 0.27197960686340544, 0.0, 39, [0.065, 0.21, 0], 0],  #Configuraciones de shooting
  [0.4124976804194961, 0.6572465438490181, 0.0, 27.58107962474939, [0.38624352818436225, 0.26484374999999993, 0.0], 0],
    [0.6583001921626154, 1.055582036963348, 0.0, 23.501513172604934, [0.686179888600995, 0.23906249999999996, 0.0], 0],
    [0.16366454479769424, 0.24739286519004644, 0.0, 31, [0.011850204914755191, 0.20390624999999998, 0.0], 0],
    [0.7487891997390527, 1.2016735866483446, 0.0, 22.07823531930968, [0.795852456887079, 0.20390624999999998, 0.0], 0],
    [0.7362258448429816, 1.1813911884373012, 0.0, 22.48457242152914, [0.7804419337105537, 0.2109375, 0.0], 0],
    [0.24619929355350015, 0.3850073564627296, 0.0, 31, [0.17303253078975322, 0.23613281249999998, 0.0], 0],
    [0.37674839363565493, 0.5989768729678078, 0.0, 28.274090032433367, [0.34154414609165884, 0.26249999999999996, 0.0], 0],
    [0.7227449944434775, 1.159652973709431, 0.0, 22.455479429195826, [0.7643857795299207, 0.21562499999999998, 0.0], 0],
    [0.7834664454444413, 1.2575783357498318, 0.0, 21.95788199776755, [0.8373493048521, 0.1875, 0.0], 0],
    [0.7114532405450406, 1.1414272157305694, 0.0, 22.83543668622876, [0.750709921765771, 0.22031249999999997, 0.0], 0],
    [0.3001435720151967, 0.47374384441906914, 0.0, 31, [0.24463829395835124, 0.25019531249999993, 0.0], 0],
    [0.7905545553225801, 1.2690102370376066, 0.0, 21.913278565591767, [0.8459753595036525, 0.1828125, 0.0], 0],
    [0.0, 0.24441214593732527, 0.0, 39, [0, 0.202, 0], 0],#Primer estado exitado
]

In [ ]:
gamma = 0
alpha = gamma  #Parametro de rescalamiento, puede tomar valores de 0 y 1. Este depende del valor de gamma, el cual nos indica la polarizacion
#La cuestion es que para tener codiciones apropiadas de reescalamiento si gamma=0 alpha=0 y si gamma = 1 alpha = 1
datosSig0EnMas = Proca.MasaEnergia_Datos(datos, gamma)
Proca.Plot_paper(datosSig0EnMas,cmap_colors=['#f0784d', '#2681ab'], colormap_resolution=50)

In [ ]:
#Conectamos [0,0,0], (sx,0,0) con [0,2,0], (0,sy,0)
#Primero vamos a resolver para cierta combinación de nodos distintas condiciones iniciales para ver como varia la masa
nodos = [0,1,0]
LambT = 0  # autointeracción: 1 (repulsive), 0 (without selinteraction), -1 (attractive)
rmax0 = 25  # iteration radius

#La configuracion general: (Ini0, Uxint, Uyint, Uzint, rmax, LambT=LambT, nodos=nodos)
#Ini0 = [p0x, p1x, p0y, p1y, p0z, p1z, u1x, u1y, u1z]  # boundary conditions list
Ini0 = [0.987,0,0.01,0,0,0,0,0,0]
Uxint = [0.1, 2]
Uyint = [0.1, 2]
Uzint = [0.1, 2]

#Corremos una función que nos da la solución refinada ya de forma directa
U0, rmax, masa, Energia = Proca.shoot_ref(Ini0, Uxint, Uyint, Uzint, rmax0, LambT=LambT, nodos=nodos,sx = True, sy = True, sz = False)
Proca.plotPerf(U0, rmax, LambT, lim=False)

In [ ]:
#shooting para encontrar configuraciones con misma masa
nodos = [0,2,0]
LambT = 0  # autointeracción: 1 (repulsive), 0 (without selinteraction), -1 (attractive)
rmax0 = 64  # iteration radius, normalmente hay que extenderlo si se aumenta el numero de nodos
Uxint = [0.1, 2]
Uyint = [0.1, 2]
Uzint = [0.1, 2]
xmax, ymax =0.25, 0.20
masa = 43.58
n = 10
config = [0.9185797787133904, 0.0, 0.0, 20, [1.0, 0.0, 0], 0] #Ground state
Proca.shoot_masa(config,n,masa,xmax0=xmax, Uxint=Uxint, Uyint=Uyint, Uzint=Uzint, rmax0=rmax0,
                               LambT=LambT, nodos=nodos,sx=True,sy=True,sz=False, tol = 0.05,kmax=30,estacionario = False)
print(config)
#n ,masa ,xmax, ymax, Uxint, Uyint, Uzint, rmax0, LambT=0, nodos=[0, 0, 0],sx=True,sy=True,sz=False, tol = 0.04,kmax=10

In [ ]:
datos_2nodos = [[0.9185797787133904, 0.0, 0.0, 20, [1.0, 0.0, 0], 0],#estado base
                [0.9132136330633764, 1.6634467703819686, 0.0, 21.991842207650343, [0.9940731498794806, 0.015625, 0.0], 0],#tal vez mal x2
                #Tal vez este mal
                [0.15258573010150975, 0.2815878211317962, 0.0, 40.6914964174692, [0.12721128979759866, 0.12968940734863282, 0.0], 0],
                [0.48813889552116335, 0.8958075984930921, 0.0, 26.136555027024684, [0.5125900322486607, 0.1689453125, 0.0], 0],
                [0.15658649550142342, 0.28908389460281236, 0.0, 39.99703306426247, [0.13166665719652304, 0.13144683837890625, 0.0], 0],
                [0.4562029263175267, 0.8381305785116547, 0.0, 26.456606204267292, [0.47599443502514893, 0.1708984375, 0.0], 0],
                [0.6003328805634534, 1.0993893209617667, 0.0, 24.04504246498925, [0.6398673939785816, 0.16015625, 0.0], 0],
                [0.3765675598217896, 0.6932442935304617, 0.0, 28.400564955961148, [0.38532668667833425, 0.1689453125, 0.0], 0],
                [0.5223177606278357, 0.9580924795575217, 0.0, 25.21388832479433, [0.5512660019032497, 0.16796875, 0.0], 0],
                [0.493117830838038, 0.9049611505132229, 0.0, 25.925808958772333, [0.518097074150944, 0.16943359375, 0.0], 0],
                [0.7716488139866747, 1.4089432551271934, 0.0, 22.220433977939855, [0.8339886319608749, 0.12109375, 0.0], 0],
                [0.6874603513143942, 1.2570111307763572, 0.0, 22.960578452599727, [0.73856102993588, 0.1455078125, 0.0], 0],
                [0.7550668353768862, 1.3791188326500077, 0.0, 22.283981420148255, [0.8151884736483047, 0.126953125, 0.0], 0],
                [0.7212815370066541, 1.3180744208274349, 0.0, 22.630887354454554, [0.7769458689771553, 0.13671875, 0.0], 0],
                [0.3129159128268042, 0.5770768724898402, 0.0, 30.459524174244734, [0.3125682798233097, 0.163726806640625, 0.0], 0],
                [0.5214894349516251, 0.9564685839755726, 0.0, 25.359425931207472, [0.5503347294741894, 0.16796875, 0.0], 0],
                [0.858034651565991, 1.5654454866589091, 0.0, 21.079109791892883, [0.931592538120589, 0.08203125, 0.0], 0],
                [0.2360598845304788, 0.4361773736783247, 0.0, 33.88628420838344, [0.2241952965781424, 0.15211868286132812, 0.0], 0],
                [0.72391993408646, 1.322711898426074, 0.0, 22.736730702080195, [0.7798478309316071, 0.13671875, 0.0], 0],
                [0.09988282108187083, 0.18277895351230455, 0.0, 69, [0.06904731687725307, 0.10205078125, 0.0], 0],
                [0.8378378858186879, 1.52762657998974, 0.0, 22.4001790454662, [0.9088981843406128, 0.091796875, 0.0], 0],
                [0.8477584555397751, 1.5454019482205583, 0.0, 22.461338087212958, [0.9201270842184941, 0.0859375, 0.0], 0],
                [0.7107216362805684, 1.2979125073465538, 0.0, 23.854410024211973, [0.7652014957709902, 0.1376953125, 0.0], 0],
                [0.8214823436479272, 1.498272477817431, 0.0, 22.396982043942643, [0.8902961392095079, 0.1015625, 0.0], 0],
                [0.7317570817479202, 1.33600158370023, 0.0, 23.494523860985208, [0.789039197462313, 0.1318359375, 0.0], 0],
                [0.7679947228614693, 1.4015933419346602, 0.0, 22.966373194803115, [0.8299798200804278, 0.12109375, 0.0], 0],
                [0.8782597775612432, 1.6004709043046472, 0.0, 22.201847751055098, [0.9545078100905329, 0.06640625, 0.0], 0],
                [0.0, 0.11516254806857777, 0.0, 66, [0.0, 0.085693359375, 0.0], 0]]#estado excitado

In [ ]:
gamma = 0
alpha = gamma  #Parametro de rescalamiento, puede tomar valores de 0 y 1. Este depende del valor de gamma, el cual nos indica la polarizacion
#La cuestion es que para tener codiciones apropiadas de reescalamiento si gamma=0 alpha=0 y si gamma = 1 alpha = 1
datosSig0EnMas = Proca.MasaEnergia_Datos(datos_2nodos, gamma)
Proca.Plot_paper(datosSig0EnMas,cmap_colors=['#f0784d', '#2681ab'], colormap_resolution=50)

In [ ]:
#Ahora toca para 0,3,0, primero comprobamos el radio
#shooting para encontrar configuraciones con misma masa
nodos = [0,3,0]
LambT = 0  # autointeracción: 1 (repulsive), 0 (without selinteraction), -1 (attractive)
rmax = 60  # iteration radius, normalmente hay que extenderlo si se aumenta el numero de nodos
Uxint = [0.05, 4]
Uyint = [0.05, 4]
Uzint = [0.01, 1]
xmax, ymax =1, 0.4
masa = 43.58
n = 5
config = [0.9185797787133904, 0.0, 0.0, 20, [1.0, 0.0, 0], 0] #Ground state
Proca.shoot_masa(config,n,masa,xmax0=xmax,ymax0 = ymax, Uxint=Uxint, Uyint=Uyint, Uzint=Uzint, rmax0=rmax,
                               LambT=LambT, nodos=nodos,sx=True,sy=True,sz=False, tol = 0.05,kmax=30,estacionario = False)
print(config)

In [ ]:
datos_3nodos =[[0.9185797787133904, 0.0, 0.0, 20, [1.0, 0.0, 0], 0], #Ground state
               [0.41823949130875915, 0.8436319894733442, 0.0, 27.678277475841103, [0.42943496989692787, 0.18798828125, 0.0], 0], #mas o menos
               [0.2031087111064113, 0.4097488279536135, 0.0, 40.38040309088324, [0.19158835894587434, 0.1374969482421875, 0.0], 0], #mas o menos
               [0.6937658111323882, 1.404364174413283, 0.0, 20.941753881306365, [0.741155196394828, 0.18046875, 0.0], 0], #radio de 4
               [0.7999196250853843, 1.6008719205596749, 0.0, 20.448362240304505, [0.8640242288768469, 0.13437500000000002, 0.0], 0], #radio 4.5
               [0.5734356796542869, 1.1623247479174195, 0.0, 22.979164438359437, [0.6035174126345187, 0.19843750000000002, 0.0], 0],
               [0.6948487078792452, 1.4000228305807267, 0.0, 21.3432295195704, [0.7431644806060196, 0.17500000000000002, 0.0], 0],
               
              [0.0, 0.06920917454019213, 0.0, 106, [0.0, 0.0484375, 0.0], 0],#excited state
              ]

In [ ]:
#Graficamos el primer estado excitado
auxx = [0.0, 0.06920917454019213, 0.0, 106, [0.0, 0.0484375, 0.0], 0]
U00 = [auxx[4][0],0,auxx[4][1],0,auxx[4][2], 0,auxx[0], 0,auxx[1],0,auxx[2],0]
Proca.plotPerf(U00,auxx[3],0)

